# Task
Simulate the 2026 World Cup, starting by filtering historical match data from January 1, 2014, recalculating Elo ratings for all teams, selecting 48 World Cup teams (including Colombia) based on these updated ratings, and then simulating both the group and knockout stages to analyze Colombia's performance.

## Update Data Filtering

### Subtask:
Filter the historical match data to include only matches played from January 1, 2014, onwards, as per the new requirement.


**Reasoning**:
The subtask requires filtering the `df_matches` DataFrame to include only matches played from January 1, 2014, onwards. This involves converting the 'date' column to datetime objects first and then applying the filter.



In [24]:
df_matches['date'] = pd.to_datetime(df_matches['date'])
df_filtered_matches = df_matches[df_matches['date'] >= '2014-01-01'].copy()
print("Filtered DataFrame head:")
print(df_filtered_matches.head())
print("\nFiltered DataFrame info:")
print(df_filtered_matches.info())

Filtered DataFrame head:
            date home_team away_team  home_score  away_score  \
39622 2014-01-01    Jordan    Kuwait           2           1   
39623 2014-01-04     Qatar    Kuwait           3           0   
39624 2014-01-04    Jordan   Bahrain           1           0   
39625 2014-01-07   Bahrain    Kuwait           0           0   
39626 2014-01-07     Qatar    Jordan           2           0   

                    tournament country  neutral  outcome  
39622  West Asian Championship   Qatar     True        2  
39623  West Asian Championship   Qatar    False        2  
39624  West Asian Championship   Qatar     True        2  
39625  West Asian Championship   Qatar     True        1  
39626  West Asian Championship   Qatar    False        2  

Filtered DataFrame info:
<class 'pandas.core.frame.DataFrame'>
Index: 11500 entries, 39622 to 51121
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   da

## Recalculate Elo Ratings

### Subtask:
Reinitialize Elo ratings for all unique teams found in the newly filtered dataset and then recalculate them by iterating through these matches chronologically.


**Reasoning**:
First, I'll extract all unique team names from both the 'home_team' and 'away_team' columns of the `df_filtered_matches` DataFrame and store them in a list. This step is crucial for initializing the Elo ratings for all participating teams.



In [25]:
unique_teams = pd.concat([df_filtered_matches['home_team'], df_filtered_matches['away_team']]).unique()
print(f"Number of unique teams: {len(unique_teams)}")
print("First 10 unique teams:")
print(unique_teams[:10])

Number of unique teams: 229
First 10 unique teams:
['Jordan' 'Qatar' 'Bahrain' 'Norway' 'Sweden' 'Poland' 'Chile' 'Oman'
 'South Korea' 'Mexico']


**Reasoning**:
Now that the unique teams have been identified, I will initialize a dictionary called `elo_ratings` where each unique team is a key and its initial Elo rating is set to 1500, as specified in the instructions.



In [26]:
initial_elo = 1500
elo_ratings = {team: initial_elo for team in unique_teams}
print(f"Initialized Elo ratings for {len(elo_ratings)} teams.")
print("Example Elo ratings for first 5 teams:")
for team in list(elo_ratings.keys())[:5]:
    print(f"{team}: {elo_ratings[team]}")

Initialized Elo ratings for 229 teams.
Example Elo ratings for first 5 teams:
Jordan: 1500
Qatar: 1500
Bahrain: 1500
Norway: 1500
Sweden: 1500


**Reasoning**:
I will define the `calculate_elo_change` function as specified in the instructions. This function will calculate the Elo rating changes for two teams based on their current Elo ratings, match scores, and a K-factor.



In [27]:
def calculate_elo_change(elo_home, elo_away, home_score, away_score, k_factor=30):
    # Calculate expected outcome
    expected_home = 1 / (1 + 10**((elo_away - elo_home) / 400))
    expected_away = 1 / (1 + 10**((elo_home - elo_away) / 400))

    # Determine actual outcome
    if home_score > away_score:
        actual_home = 1  # Home team wins
        actual_away = 0
    elif home_score < away_score:
        actual_home = 0  # Away team wins
        actual_away = 1
    else:
        actual_home = 0.5  # Draw
        actual_away = 0.5

    # Calculate Elo change
    change_home = k_factor * (actual_home - expected_home)
    change_away = k_factor * (actual_away - expected_away)

    return change_home, change_away

print("Defined calculate_elo_change function.")

Defined calculate_elo_change function.


**Reasoning**:
Now, I will iterate through each match in the `df_filtered_matches` DataFrame chronologically, calculate Elo changes using the `calculate_elo_change` function, update the `elo_ratings` dictionary, and record the 'before' and 'after' Elo ratings in new columns of the DataFrame as per the subtask instructions. Since the dataframe is large, using `apply` function to create these columns might be memory efficient instead of row-wise append.



In [28]:
df_filtered_matches = df_filtered_matches.sort_values(by='date').reset_index(drop=True)

# Prepare lists to store new column data
home_elo_before = []
away_elo_before = []
home_elo_after = []
away_elo_after = []

for index, row in df_filtered_matches.iterrows():
    home_team = row['home_team']
    away_team = row['away_team']
    home_score = row['home_score']
    away_score = row['away_score']

    # Retrieve current Elo ratings
    elo_home = elo_ratings.get(home_team, initial_elo) # Use .get() with default initial_elo if team not in dict (shouldn't happen here)
    elo_away = elo_ratings.get(away_team, initial_elo)

    # Store 'before match' Elo ratings
    home_elo_before.append(elo_home)
    away_elo_before.append(elo_away)

    # Calculate Elo change
    change_home, change_away = calculate_elo_change(elo_home, elo_away, home_score, away_score)

    # Update Elo ratings
    new_elo_home = elo_home + change_home
    new_elo_away = elo_away + change_away

    elo_ratings[home_team] = new_elo_home
    elo_ratings[away_team] = new_elo_away

    # Store 'after match' Elo ratings
    home_elo_after.append(new_elo_home)
    away_elo_after.append(new_elo_away)

# Add the new columns to the DataFrame
df_filtered_matches['home_team_elo_before'] = home_elo_before
df_filtered_matches['away_team_elo_before'] = away_elo_before
df_filtered_matches['home_team_elo_after'] = home_elo_after
df_filtered_matches['away_team_elo_after'] = away_elo_after

print("Elo ratings recalculated and updated in df_filtered_matches.")
print("DataFrame head with new Elo columns:")
print(df_filtered_matches[['date', 'home_team', 'away_team', 'home_team_elo_before', 'away_team_elo_before', 'home_team_elo_after', 'away_team_elo_after']].head())

Elo ratings recalculated and updated in df_filtered_matches.
DataFrame head with new Elo columns:
        date home_team away_team  home_team_elo_before  away_team_elo_before  \
0 2014-01-01    Jordan    Kuwait             1500.0000             1500.0000   
1 2014-01-04     Qatar    Kuwait             1500.0000             1485.0000   
2 2014-01-04    Jordan   Bahrain             1515.0000             1500.0000   
3 2014-01-07   Bahrain    Kuwait             1485.6472             1470.6472   
4 2014-01-07     Qatar    Jordan             1514.3528             1529.3528   

   home_team_elo_after  away_team_elo_after  
0            1515.0000            1485.0000  
1            1514.3528            1470.6472  
2            1529.3528            1485.6472  
3            1485.0000            1471.2944  
4            1530.0000            1513.7056  


## Select World Cup Teams (Including Colombia)

### Subtask:
Select the 48 teams for the World Cup 2026. This selection will ensure Colombia is included, and the remaining teams will be chosen based on their updated Elo ratings.


**Reasoning**:
I will convert the `elo_ratings` dictionary into a Pandas Series, sort it in descending order, ensure 'Colombia' is included, and then select the top 47 teams based on their Elo ratings to form the final 48 World Cup teams.



In [29]:
import pandas as pd

# 1. Convert elo_ratings dictionary into a Pandas Series
elo_series = pd.Series(elo_ratings)

# 2. Sort this Series in descending order
sorted_elo_series = elo_series.sort_values(ascending=False)

# 3. Define 'Colombia' as a mandatory participant and remove it from the ranked list
mandatory_team = 'Colombia'
if mandatory_team in sorted_elo_series:
    remaining_teams_elo = sorted_elo_series.drop(mandatory_team)
else:
    remaining_teams_elo = sorted_elo_series.copy()
    print(f"Warning: {mandatory_team} not found in Elo ratings. It will be added manually.")

# 4. Select the top 47 teams from the remaining ranked list
selected_47_teams = remaining_teams_elo.head(47).index.tolist()

# 5. Combine 'Colombia' with these 47 teams to form the final list of 48 World Cup teams
world_cup_teams = [mandatory_team] + selected_47_teams

# 6. Print the total number of selected teams and the list of these teams
print(f"Total number of World Cup teams: {len(world_cup_teams)}")
print("Selected World Cup Teams:")
for i, team in enumerate(world_cup_teams):
    print(f"{i+1}. {team}")


Total number of World Cup teams: 48
Selected World Cup Teams:
1. Colombia
2. Spain
3. Argentina
4. France
5. Japan
6. England
7. Morocco
8. Brazil
9. Portugal
10. Netherlands
11. South Korea
12. Germany
13. Senegal
14. Iran
15. Uzbekistan
16. Croatia
17. Mexico
18. Uruguay
19. Australia
20. Algeria
21. Italy
22. Ecuador
23. Denmark
24. United States
25. Switzerland
26. Canada
27. Norway
28. Russia
29. Belgium
30. Egypt
31. Turkey
32. Tunisia
33. Austria
34. Panama
35. Iraq
36. Ivory Coast
37. Saudi Arabia
38. Ukraine
39. Poland
40. Mali
41. Nigeria
42. Greece
43. Costa Rica
44. Serbia
45. DR Congo
46. South Africa
47. United Arab Emirates
48. Paraguay


## Define World Cup 2026 Structure

### Subtask:
Reconfirm the structure of the World Cup 2026, including the number of groups and the progression rules for both group and knockout stages, as this forms the framework for the simulation.


### World Cup 2026 Structure and Progression Rules

**Group Stage:**
*   **Number of Groups:** 12 groups
*   **Teams per Group:** 4 teams
*   **Progression Rules:**
    *   Top 2 teams from each of the 12 groups (24 teams) automatically advance.
    *   8 best third-placed teams across all groups also qualify.
    *   Total of 32 teams advancing to the knockout stage.

**Knockout Stage:**
*   **Starting Round:** Round of 32
*   **Progression Rules:** Standard single-elimination rules apply.

**Reasoning**:
To make the defined World Cup structure parameters accessible for the simulation, I will store them as Python variables.



In [30]:
num_groups = 12
teams_per_group = 4
group_stage_progression_rules = [
    "Top 2 teams from each of the 12 groups (24 teams) automatically advance.",
    "8 best third-placed teams across all groups also qualify.",
    "Total of 32 teams advancing to the knockout stage."
]
knockout_stage_progression_rules = [
    "Round of 32",
    "Round of 16",
    "Quarter-finals",
    "Semi-finals",
    "Final",
    "Standard single-elimination rules apply."
]

print(f"Number of World Cup groups: {num_groups}")
print(f"Teams per group: {teams_per_group}")
print("Group stage progression rules defined.")
print("Knockout stage progression rules defined.")

Number of World Cup groups: 12
Teams per group: 4
Group stage progression rules defined.
Knockout stage progression rules defined.


## Simulate Group Stage Matches

### Subtask:
Simulate all matches in the group stage using the newly calculated Elo ratings, determine group standings, and identify which teams advance to the knockout rounds.


**Reasoning**:
I will define the `simulate_match` function as specified in the instructions. This function will take the Elo ratings of two teams and return the winner and simulated scores based on a probabilistic approach.



In [31]:
import numpy as np

def simulate_match(elo_team1, elo_team2):
    # Probability of Team 1 winning based on Elo difference
    prob_team1_wins = 1 / (1 + 10**((elo_team2 - elo_team1) / 400))

    # Simulate outcome
    rand = np.random.rand()
    if rand < prob_team1_wins * 0.7:  # Adjust probabilities for win/draw bias
        outcome = 1 # Team 1 wins
    elif rand < prob_team1_wins * 0.7 + (1 - prob_team1_wins) * 0.2: # Draw probability
        outcome = 0 # Draw
    else:
        outcome = -1 # Team 2 wins

    # Simulate scores (simplified for demonstration)
    # Adjust these values for more realistic score distributions if needed
    if outcome == 1: # Team 1 wins
        score_team1 = np.random.randint(1, 4)
        score_team2 = np.random.randint(0, score_team1)
    elif outcome == -1: # Team 2 wins
        score_team2 = np.random.randint(1, 4)
        score_team1 = np.random.randint(0, score_team2)
    else: # Draw
        score_team1 = np.random.randint(0, 3)
        score_team2 = score_team1

    return outcome, score_team1, score_team2

print("Defined simulate_match function.")

Defined simulate_match function.


**Reasoning**:
Now that the `simulate_match` function is defined, I will randomly divide the `world_cup_teams` into 12 groups of 4 teams each as per the instructions. This involves shuffling the teams and then splitting them into groups.



In [32]:
import random

# Shuffle the world_cup_teams to ensure random grouping
random.shuffle(world_cup_teams)

# Divide teams into 12 groups of 4
groups = {}
team_index = 0
for i in range(num_groups):
    group_name = f"Group {chr(65 + i)}" # A, B, C, ...
    groups[group_name] = world_cup_teams[team_index : team_index + teams_per_group]
    team_index += teams_per_group

print(f"Successfully divided {len(world_cup_teams)} teams into {num_groups} groups:")
for group_name, teams in groups.items():
    print(f"  {group_name}: {teams}")

Successfully divided 48 teams into 12 groups:
  Group A: ['Belgium', 'Iran', 'Mexico', 'Panama']
  Group B: ['Russia', 'Ukraine', 'Denmark', 'United Arab Emirates']
  Group C: ['South Korea', 'Colombia', 'England', 'Turkey']
  Group D: ['Serbia', 'Ecuador', 'Italy', 'Greece']
  Group E: ['Uruguay', 'South Africa', 'Egypt', 'Saudi Arabia']
  Group F: ['Paraguay', 'Portugal', 'Morocco', 'United States']
  Group G: ['Mali', 'DR Congo', 'Costa Rica', 'Ivory Coast']
  Group H: ['Spain', 'Senegal', 'Croatia', 'Tunisia']
  Group I: ['Australia', 'Uzbekistan', 'Brazil', 'Algeria']
  Group J: ['Germany', 'Iraq', 'Austria', 'Argentina']
  Group K: ['Nigeria', 'Netherlands', 'Canada', 'Poland']
  Group L: ['Japan', 'Norway', 'Switzerland', 'France']


**Reasoning**:
I will define a function `simulate_group_matches` that iterates through all teams within a given group, simulates matches between each pair of teams using the `simulate_match` function, and initializes a dictionary to track each team's performance for group standings.



In [33]:
def simulate_group_matches(group_teams, elo_ratings_dict):
    group_results = []
    # Initialize standings for teams in the group
    group_standings = {team: {'points': 0, 'wins': 0, 'draws': 0, 'losses': 0, 'goals_for': 0, 'goals_against': 0, 'goal_difference': 0} for team in group_teams}

    # Each team plays every other team once (6 matches per group of 4 teams)
    for i in range(len(group_teams)):
        for j in range(i + 1, len(group_teams)):
            team1 = group_teams[i]
            team2 = group_teams[j]

            elo_team1 = elo_ratings_dict.get(team1, initial_elo)
            elo_team2 = elo_ratings_dict.get(team2, initial_elo)

            outcome, score_team1, score_team2 = simulate_match(elo_team1, elo_team2)

            # Record match result
            group_results.append({
                'team1': team1,
                'team2': team2,
                'score_team1': score_team1,
                'score_team2': score_team2,
                'outcome': outcome # 1 for team1 win, -1 for team2 win, 0 for draw
            })

            # Update standings
            if outcome == 1: # Team 1 wins
                group_standings[team1]['points'] += 3
                group_standings[team1]['wins'] += 1
                group_standings[team2]['losses'] += 1
            elif outcome == -1: # Team 2 wins
                group_standings[team2]['points'] += 3
                group_standings[team2]['wins'] += 1
                group_standings[team1]['losses'] += 1
            else: # Draw
                group_standings[team1]['points'] += 1
                group_standings[team2]['points'] += 1
                group_standings[team1]['draws'] += 1
                group_standings[team2]['draws'] += 1

            group_standings[team1]['goals_for'] += score_team1
            group_standings[team1]['goals_against'] += score_team2
            group_standings[team1]['goal_difference'] += (score_team1 - score_team2)

            group_standings[team2]['goals_for'] += score_team2
            group_standings[team2]['goals_against'] += score_team1
            group_standings[team2]['goal_difference'] += (score_team2 - score_team1)

    return group_standings

print("Defined simulate_group_matches function.")

Defined simulate_group_matches function.


**Reasoning**:
I will now iterate through each of the previously created groups, simulate their matches using the `simulate_group_matches` function, and store the calculated group standings for each group.



In [34]:
all_group_standings = {}
for group_name, teams in groups.items():
    print(f"\nSimulating matches for {group_name}:")
    standings = simulate_group_matches(teams, elo_ratings)
    all_group_standings[group_name] = standings

    print(f"Standings for {group_name}:")
    # Sort standings for better readability
    sorted_standings = sorted(standings.items(), key=lambda item: (item[1]['points'], item[1]['goal_difference'], item[1]['goals_for']), reverse=True)
    for team, stats in sorted_standings:
        print(f"  {team}: Points={stats['points']}, GD={stats['goal_difference']}, GF={stats['goals_for']}")

print("\nAll group stage matches simulated and standings calculated.")


Simulating matches for Group A:
Standings for Group A:
  Panama: Points=9, GD=4, GF=4
  Iran: Points=3, GD=0, GF=5
  Belgium: Points=3, GD=-2, GF=3
  Mexico: Points=3, GD=-2, GF=2

Simulating matches for Group B:
Standings for Group B:
  United Arab Emirates: Points=9, GD=5, GF=7
  Ukraine: Points=6, GD=0, GF=5
  Denmark: Points=3, GD=0, GF=5
  Russia: Points=0, GD=-5, GF=1

Simulating matches for Group C:
Standings for Group C:
  England: Points=6, GD=3, GF=6
  South Korea: Points=6, GD=2, GF=5
  Colombia: Points=6, GD=-1, GF=3
  Turkey: Points=0, GD=-4, GF=1

Simulating matches for Group D:
Standings for Group D:
  Italy: Points=9, GD=6, GF=6
  Serbia: Points=3, GD=0, GF=3
  Greece: Points=3, GD=-2, GF=2
  Ecuador: Points=3, GD=-4, GF=2

Simulating matches for Group E:
Standings for Group E:
  Uruguay: Points=6, GD=2, GF=6
  Saudi Arabia: Points=6, GD=2, GF=5
  South Africa: Points=4, GD=-1, GF=2
  Egypt: Points=1, GD=-3, GF=1

Simulating matches for Group F:
Standings for Group F:


**Reasoning**:
Now that all group matches have been simulated and standings calculated, I will identify the top 2 teams from each group, as they automatically qualify for the knockout stage.



In [35]:
top_two_qualifiers = []
for group_name, standings in all_group_standings.items():
    # Sort teams within each group by points, then goal difference, then goals for
    sorted_group_teams = sorted(standings.items(), key=lambda item: (item[1]['points'], item[1]['goal_difference'], item[1]['goals_for']), reverse=True)

    # The top 2 teams qualify directly
    top_two_qualifiers.extend([team for team, stats in sorted_group_teams[:2]])

print(f"Total {len(top_two_qualifiers)} teams qualified directly (top 2 from each group).")
print("Directly qualified teams:")
for i, team in enumerate(top_two_qualifiers):
    print(f"  {i+1}. {team}")


Total 24 teams qualified directly (top 2 from each group).
Directly qualified teams:
  1. Panama
  2. Iran
  3. United Arab Emirates
  4. Ukraine
  5. England
  6. South Korea
  7. Italy
  8. Serbia
  9. Uruguay
  10. Saudi Arabia
  11. Portugal
  12. Morocco
  13. Costa Rica
  14. Mali
  15. Senegal
  16. Croatia
  17. Algeria
  18. Uzbekistan
  19. Germany
  20. Argentina
  21. Poland
  22. Nigeria
  23. Japan
  24. France


**Reasoning**:
I will now identify all the third-placed teams from each group, sort them based on the specified criteria (points, goal difference, goals for), and select the top 8 to join the `top_two_qualifiers` for the knockout stage.



In [36]:
third_placed_teams = []
for group_name, standings in all_group_standings.items():
    sorted_group_teams = sorted(standings.items(), key=lambda item: (item[1]['points'], item[1]['goal_difference'], item[1]['goals_for']), reverse=True)
    # The third team in the sorted list is the third-placed team
    if len(sorted_group_teams) > 2:
        team_name, stats = sorted_group_teams[2]
        third_placed_teams.append({'team': team_name, **stats})

# Sort third-placed teams by points, then goal difference, then goals for
sorted_third_placed = sorted(third_placed_teams, key=lambda x: (x['points'], x['goal_difference'], x['goals_for']), reverse=True)

# Select the 8 best third-placed teams
best_third_placed_qualifiers = [team['team'] for team in sorted_third_placed[:8]]

print(f"Total {len(best_third_placed_qualifiers)} best third-placed teams qualified.")
print("Best third-placed qualified teams:")
for i, team in enumerate(best_third_placed_qualifiers):
    print(f"  {i+1}. {team}")

# Combine with top_two_qualifiers for the final 32 teams
all_qualified_teams = top_two_qualifiers + best_third_placed_qualifiers
print(f"\nTotal teams advancing to knockout stage: {len(all_qualified_teams)}")
print("All qualified teams:")
print(all_qualified_teams)

Total 8 best third-placed teams qualified.
Best third-placed qualified teams:
  1. Colombia
  2. Switzerland
  3. Brazil
  4. DR Congo
  5. South Africa
  6. Denmark
  7. Netherlands
  8. Paraguay

Total teams advancing to knockout stage: 32
All qualified teams:
['Panama', 'Iran', 'United Arab Emirates', 'Ukraine', 'England', 'South Korea', 'Italy', 'Serbia', 'Uruguay', 'Saudi Arabia', 'Portugal', 'Morocco', 'Costa Rica', 'Mali', 'Senegal', 'Croatia', 'Algeria', 'Uzbekistan', 'Germany', 'Argentina', 'Poland', 'Nigeria', 'Japan', 'France', 'Colombia', 'Switzerland', 'Brazil', 'DR Congo', 'South Africa', 'Denmark', 'Netherlands', 'Paraguay']


## Simulate Knockout Stage Matches

### Subtask:
Simulate the matches from the Round of 32 up to the final, based on the outcomes of the group stage and using the updated Elo ratings for predictions.


**Reasoning**:
I will define the `simulate_knockout_match` function as specified, which will take two team names and the `elo_ratings` dictionary, calculate win probabilities based on Elo ratings, and return the winning team.



In [37]:
def simulate_knockout_match(team1_name, team2_name, elo_ratings_dict):
    elo_team1 = elo_ratings_dict.get(team1_name, initial_elo)
    elo_team2 = elo_ratings_dict.get(team2_name, initial_elo)

    # Probability of Team 1 winning based on Elo difference
    prob_team1_wins = 1 / (1 + 10**((elo_team2 - elo_team1) / 400))

    # Simulate outcome - no draws in knockout, only a winner
    rand = np.random.rand()
    if rand < prob_team1_wins:
        return team1_name
    else:
        return team2_name

print("Defined simulate_knockout_match function.")

Defined simulate_knockout_match function.


**Reasoning**:
Now that the `simulate_knockout_match` function is defined and we have the 32 qualified teams, I will set up the initial pairings for the Round of 32 by shuffling the qualified teams and then simulate each round of the knockout stage (Round of 32, Round of 16, Quarter-finals, Semi-finals, and Final) to determine the ultimate winner of the World Cup.



In [38]:
import random

# Shuffle the qualified teams for random pairings in Round of 32
random.shuffle(all_qualified_teams)

knockout_results = {}
current_round_teams = list(all_qualified_teams)

round_names = ['Round of 32', 'Round of 16', 'Quarter-finals', 'Semi-finals', 'Final']

for round_name in round_names:
    print(f"\n--- {round_name} ---")
    winners_of_round = []
    matches_this_round = []

    # Pair teams for the current round
    # Assuming an even number of teams for pairing
    for i in range(0, len(current_round_teams), 2):
        team1 = current_round_teams[i]
        team2 = current_round_teams[i+1]

        winner = simulate_knockout_match(team1, team2, elo_ratings)
        winners_of_round.append(winner)

        loser = team1 if winner == team2 else team2
        matches_this_round.append((team1, team2, winner))

        print(f"  {team1} vs {team2} -> Winner: {winner}")

    knockout_results[round_name] = winners_of_round
    current_round_teams = winners_of_round

    if len(current_round_teams) == 1:
        final_winner = current_round_teams[0]
        print(f"\n--- World Cup 2026 Winner: {final_winner} ---")
        break

print("\nKnockout stage simulation complete.")


--- Round of 32 ---
  Morocco vs Argentina -> Winner: Morocco
  France vs Netherlands -> Winner: France
  Germany vs Uzbekistan -> Winner: Germany
  DR Congo vs Ukraine -> Winner: Ukraine
  Switzerland vs England -> Winner: England
  Serbia vs Italy -> Winner: Serbia
  Algeria vs Nigeria -> Winner: Algeria
  Iran vs Colombia -> Winner: Colombia
  Brazil vs Japan -> Winner: Brazil
  Portugal vs Paraguay -> Winner: Portugal
  Costa Rica vs Mali -> Winner: Mali
  Poland vs Denmark -> Winner: Denmark
  Uruguay vs Panama -> Winner: Uruguay
  United Arab Emirates vs Senegal -> Winner: United Arab Emirates
  Saudi Arabia vs South Africa -> Winner: Saudi Arabia
  Croatia vs South Korea -> Winner: Croatia

--- Round of 16 ---
  Morocco vs France -> Winner: Morocco
  Germany vs Ukraine -> Winner: Ukraine
  England vs Serbia -> Winner: Serbia
  Algeria vs Colombia -> Winner: Algeria
  Brazil vs Portugal -> Winner: Brazil
  Mali vs Denmark -> Winner: Denmark
  Uruguay vs United Arab Emirates -> W

## Analyze Colombia's Performance

### Subtask:
Analyze the results of the full tournament simulation to determine how far Colombia advanced and their approximate final standing.


## Summary:

### Q&A
**How far did Colombia advance in the simulated World Cup 2026, and what was their approximate final standing?**
Colombia successfully advanced from the group stage as one of the 8 best third-placed teams. They won their Round of 32 match against Iran but were subsequently eliminated in the Round of 16 after losing to Algeria. This places Colombia among the teams that reached the Round of 16.

### Data Analysis Key Findings
*   Historical match data was filtered to include 11,500 matches played from January 1, 2014, onwards.
*   Elo ratings were re-calculated for 229 unique teams based on this filtered data, starting with an initial rating of 1500 for each team.
*   48 teams were selected for the World Cup 2026 simulation, including Colombia as a mandatory participant and 47 other teams chosen based on their updated Elo ratings (Colombia was listed first among the selected teams).
*   The World Cup 2026 structure was defined with 12 groups of 4 teams. The top 2 teams from each group (24 teams) and the 8 best third-placed teams would advance to a Round of 32 knockout stage.
*   In the simulated group stage, Colombia finished third in Group C with 6 points, a goal difference of -1, and 3 goals for.
*   Colombia successfully qualified for the knockout stage as one of the 8 best third-placed teams, making up the total of 32 teams.
*   In the knockout stage simulation, Colombia won its Round of 32 match against Iran.
*   Colombia was eliminated in the Round of 16 after losing to Algeria.
*   The simulation concluded with Morocco winning the World Cup 2026, defeating Uruguay in the final.

### Insights or Next Steps
*   Colombia's performance was competitive, successfully navigating the group stage and securing a win in the first knockout round, indicating a solid, though not top-tier, performance.
*   To gain a more statistically robust understanding of team performance and tournament outcomes, it would be beneficial to run this full simulation multiple times (e.g., 1,000 or 10,000 iterations) and then average the results to account for the stochastic nature of the match simulations.


# Task
Simulate the 2026 World Cup, starting by filtering historical match data from January 1, 2018, recalculating Elo ratings for all teams, selecting 48 World Cup teams (including Colombia) based on these updated ratings, and then simulating both the group and knockout stages to analyze Colombia's performance.

## Update Data Filtering to 2018

### Subtask:
Filter the historical match data to include only matches played from January 1, 2018, onwards, as per the new requirement.


**Reasoning**:
The subtask requires filtering the `df_matches` DataFrame to include only matches played from January 1, 2018, onwards. The 'date' column has already been converted to datetime objects in a previous step, so I will apply the filter directly and then display the head and info of the new DataFrame.



In [39]:
df_filtered_matches_2018 = df_matches[df_matches['date'] >= '2018-01-01'].copy()
print("Filtered DataFrame head (2018 onwards):")
print(df_filtered_matches_2018.head())
print("\nFiltered DataFrame info (2018 onwards):")
print(df_filtered_matches_2018.info())

Filtered DataFrame head (2018 onwards):
            date home_team             away_team  home_score  away_score  \
43327 2018-01-02      Iraq  United Arab Emirates           0           0   
43328 2018-01-02      Oman               Bahrain           1           0   
43329 2018-01-05      Oman  United Arab Emirates           0           0   
43330 2018-01-07   Estonia                Sweden           1           1   
43331 2018-01-11   Finland                Jordan           2           1   

      tournament               country  neutral  outcome  
43327   Gulf Cup                Kuwait     True        1  
43328   Gulf Cup                Kuwait     True        2  
43329   Gulf Cup                Kuwait     True        1  
43330   Friendly  United Arab Emirates     True        1  
43331   Friendly  United Arab Emirates     True        2  

Filtered DataFrame info (2018 onwards):
<class 'pandas.core.frame.DataFrame'>
Index: 7795 entries, 43327 to 51121
Data columns (total 9 columns):
 #

## Recalculate Elo Ratings

### Subtask:
Reinitialize Elo ratings for all unique teams found in the newly filtered dataset and then recalculate them by iterating through these matches chronologically.


**Reasoning**:
First, I'll extract all unique team names from both the 'home_team' and 'away_team' columns of the `df_filtered_matches_2018` DataFrame and store them in a list. This is necessary to initialize the Elo ratings for all participating teams in this new filtered dataset.



In [40]:
unique_teams_2018 = pd.concat([df_filtered_matches_2018['home_team'], df_filtered_matches_2018['away_team']]).unique()
print(f"Number of unique teams (2018 onwards): {len(unique_teams_2018)}")
print("First 10 unique teams (2018 onwards):")
print(unique_teams_2018[:10])

Number of unique teams (2018 onwards): 229
First 10 unique teams (2018 onwards):
['Iraq' 'Oman' 'Estonia' 'Finland' 'Indonesia' 'South Korea'
 'United States' 'Jamaica' 'Azerbaijan' 'Mexico']


**Reasoning**:
Now that the unique teams from the 2018 filtered dataset have been identified, I will initialize a dictionary called `elo_ratings_2018` where each unique team is a key and its initial Elo rating is set to 1500, as specified in the instructions.



In [41]:
initial_elo = 1500
elo_ratings_2018 = {team: initial_elo for team in unique_teams_2018}
print(f"Initialized Elo ratings for {len(elo_ratings_2018)} teams (2018 onwards).")
print("Example Elo ratings for first 5 teams (2018 onwards):")
for team in list(elo_ratings_2018.keys())[:5]:
    print(f"{team}: {elo_ratings_2018[team]}")

Initialized Elo ratings for 229 teams (2018 onwards).
Example Elo ratings for first 5 teams (2018 onwards):
Iraq: 1500
Oman: 1500
Estonia: 1500
Finland: 1500
Indonesia: 1500


**Reasoning**:
I will sort the `df_filtered_matches_2018` DataFrame by date, then iterate through each match to recalculate Elo ratings, updating the `elo_ratings_2018` dictionary and adding 'before' and 'after' Elo ratings as new columns to the DataFrame.



In [42]:
df_filtered_matches_2018 = df_filtered_matches_2018.sort_values(by='date').reset_index(drop=True)

# Prepare lists to store new column data
home_elo_before_2018 = []
away_elo_before_2018 = []
home_elo_after_2018 = []
away_elo_after_2018 = []

for index, row in df_filtered_matches_2018.iterrows():
    home_team = row['home_team']
    away_team = row['away_team']
    home_score = row['home_score']
    away_score = row['away_score']

    # Retrieve current Elo ratings
    elo_home = elo_ratings_2018.get(home_team, initial_elo)
    elo_away = elo_ratings_2018.get(away_team, initial_elo)

    # Store 'before match' Elo ratings
    home_elo_before_2018.append(elo_home)
    away_elo_before_2018.append(elo_away)

    # Calculate Elo change using the previously defined function
    change_home, change_away = calculate_elo_change(elo_home, elo_away, home_score, away_score)

    # Update Elo ratings
    new_elo_home = elo_home + change_home
    new_elo_away = elo_away + change_away

    elo_ratings_2018[home_team] = new_elo_home
    elo_ratings_2018[away_team] = new_elo_away

    # Store 'after match' Elo ratings
    home_elo_after_2018.append(new_elo_home)
    away_elo_after_2018.append(new_elo_away)

# Add the new columns to the DataFrame
df_filtered_matches_2018['home_team_elo_before_2018'] = home_elo_before_2018
df_filtered_matches_2018['away_team_elo_before_2018'] = away_elo_before_2018
df_filtered_matches_2018['home_team_elo_after_2018'] = home_elo_after_2018
df_filtered_matches_2018['away_team_elo_after_2018'] = away_elo_after_2018

print("Elo ratings recalculated and updated in df_filtered_matches_2018.")
print("DataFrame head with new Elo columns (2018 onwards):")
print(df_filtered_matches_2018[['date', 'home_team', 'away_team', 'home_team_elo_before_2018', 'away_team_elo_before_2018', 'home_team_elo_after_2018', 'away_team_elo_after_2018']].head())

Elo ratings recalculated and updated in df_filtered_matches_2018.
DataFrame head with new Elo columns (2018 onwards):
        date home_team             away_team  home_team_elo_before_2018  \
0 2018-01-02      Iraq  United Arab Emirates                     1500.0   
1 2018-01-02      Oman               Bahrain                     1500.0   
2 2018-01-05      Oman  United Arab Emirates                     1515.0   
3 2018-01-07   Estonia                Sweden                     1500.0   
4 2018-01-11   Finland                Jordan                     1500.0   

   away_team_elo_before_2018  home_team_elo_after_2018  \
0                     1500.0                 1500.0000   
1                     1500.0                 1515.0000   
2                     1500.0                 1514.3528   
3                     1500.0                 1500.0000   
4                     1500.0                 1515.0000   

   away_team_elo_after_2018  
0                 1500.0000  
1                 1485

## Select World Cup Teams (Including Colombia)

### Subtask:
Select the 48 teams for the World Cup 2026. This selection will ensure Colombia is included, and the remaining teams will be chosen based on their updated Elo ratings.


**Reasoning**:
I will convert the `elo_ratings_2018` dictionary into a Pandas Series, sort it in descending order, ensure 'Colombia' is included, and then select the top 47 teams based on their Elo ratings to form the final 48 World Cup teams, as per the instructions.



In [43]:
import pandas as pd

# 1. Convert elo_ratings_2018 dictionary into a Pandas Series
elo_series_2018 = pd.Series(elo_ratings_2018)

# 2. Sort this Series in descending order
sorted_elo_series_2018 = elo_series_2018.sort_values(ascending=False)

# 3. Define 'Colombia' as a mandatory participant and remove it from the ranked list
mandatory_team = 'Colombia'
if mandatory_team in sorted_elo_series_2018:
    remaining_teams_elo_2018 = sorted_elo_series_2018.drop(mandatory_team)
else:
    remaining_teams_elo_2018 = sorted_elo_series_2018.copy()
    print(f"Warning: {mandatory_team} not found in Elo ratings. It will be added manually.")

# 4. Select the top 47 teams from the remaining ranked list
selected_47_teams_2018 = remaining_teams_elo_2018.head(47).index.tolist()

# 5. Combine 'Colombia' with these 47 teams to form the final list of 48 World Cup teams
world_cup_teams_2018 = [mandatory_team] + selected_47_teams_2018

# 6. Print the total number of selected teams and the list of these teams
print(f"Total number of World Cup teams (2018 onwards): {len(world_cup_teams_2018)}")
print("Selected World Cup Teams (2018 onwards):")
for i, team in enumerate(world_cup_teams_2018):
    print(f"  {i+1}. {team}")

Total number of World Cup teams (2018 onwards): 48
Selected World Cup Teams (2018 onwards):
  1. Colombia
  2. Spain
  3. Argentina
  4. Morocco
  5. France
  6. Japan
  7. England
  8. Portugal
  9. Senegal
  10. South Korea
  11. Brazil
  12. Uzbekistan
  13. Netherlands
  14. Iran
  15. Algeria
  16. Germany
  17. Croatia
  18. Australia
  19. Mexico
  20. Norway
  21. United States
  22. Canada
  23. Italy
  24. Denmark
  25. Uruguay
  26. Switzerland
  27. Ecuador
  28. Egypt
  29. Russia
  30. Tunisia
  31. Turkey
  32. Belgium
  33. Austria
  34. Panama
  35. Iraq
  36. Ivory Coast
  37. Mali
  38. Saudi Arabia
  39. South Africa
  40. DR Congo
  41. Nigeria
  42. Costa Rica
  43. Greece
  44. United Arab Emirates
  45. Cape Verde
  46. Ukraine
  47. Poland
  48. Burkina Faso


## Simulate Group Stage Matches

### Subtask:
Simulate all matches in the group stage using the newly calculated Elo ratings, determine group standings, and identify which teams advance to the knockout rounds.


**Reasoning**:
I will simulate the group stage for the 2018 filtered data. This involves shuffling the teams, dividing them into groups, simulating matches for each group, calculating standings, and then identifying both the top two and best third-placed qualifiers to form the final list of teams advancing to the knockout stage.



In [44]:
import random

# 1. Randomly shuffle the world_cup_teams_2018 list
random.shuffle(world_cup_teams_2018)

# 2. Divide into groups
groups_2018 = {}
team_index = 0
for i in range(num_groups):
    group_name = f"Group {chr(65 + i)}"
    groups_2018[group_name] = world_cup_teams_2018[team_index : team_index + teams_per_group]
    team_index += teams_per_group

print(f"Successfully divided {len(world_cup_teams_2018)} teams into {num_groups} groups (2018 onwards):")
for group_name, teams in groups_2018.items():
    print(f"  {group_name}: {teams}")

# 3. Simulate group matches and store standings
all_group_standings_2018 = {}
for group_name, teams in groups_2018.items():
    print(f"\nSimulating matches for {group_name} (2018 onwards):")
    standings = simulate_group_matches(teams, elo_ratings_2018)
    all_group_standings_2018[group_name] = standings

    # 4. Print sorted group standings
    print(f"Standings for {group_name} (2018 onwards):")
    sorted_standings = sorted(standings.items(), key=lambda item: (item[1]['points'], item[1]['goal_difference'], item[1]['goals_for']), reverse=True)
    for team, stats in sorted_standings:
        print(f"  {team}: Points={stats['points']}, GD={stats['goal_difference']}, GF={stats['goals_for']}")

print("\nAll group stage matches simulated and standings calculated (2018 onwards).")

# 5. Identify top 2 qualifiers
top_two_qualifiers_2018 = []
for group_name, standings in all_group_standings_2018.items():
    sorted_group_teams = sorted(standings.items(), key=lambda item: (item[1]['points'], item[1]['goal_difference'], item[1]['goals_for']), reverse=True)
    top_two_qualifiers_2018.extend([team for team, stats in sorted_group_teams[:2]])

print(f"\nTotal {len(top_two_qualifiers_2018)} teams qualified directly (top 2 from each group, 2018 onwards).")
print("Directly qualified teams (2018 onwards):")
for i, team in enumerate(top_two_qualifiers_2018):
    print(f"  {i+1}. {team}")

# 6. Identify all third-placed teams
third_placed_teams_2018 = []
for group_name, standings in all_group_standings_2018.items():
    sorted_group_teams = sorted(standings.items(), key=lambda item: (item[1]['points'], item[1]['goal_difference'], item[1]['goals_for']), reverse=True)
    if len(sorted_group_teams) > 2:
        team_name, stats = sorted_group_teams[2]
        third_placed_teams_2018.append({'team': team_name, **stats})

# 7. Sort third-placed teams and select top 8
sorted_third_placed_2018 = sorted(third_placed_teams_2018, key=lambda x: (x['points'], x['goal_difference'], x['goals_for']), reverse=True)
best_third_placed_qualifiers_2018 = [team['team'] for team in sorted_third_placed_2018[:8]]

print(f"\nTotal {len(best_third_placed_qualifiers_2018)} best third-placed teams qualified (2018 onwards).")
print("Best third-placed qualified teams (2018 onwards):")
for i, team in enumerate(best_third_placed_qualifiers_2018):
    print(f"  {i+1}. {team}")

# 8. Combine for final 32 teams
all_qualified_teams_2018 = top_two_qualifiers_2018 + best_third_placed_qualifiers_2018
print(f"\nTotal teams advancing to knockout stage (2018 onwards): {len(all_qualified_teams_2018)}")
print("All qualified teams (2018 onwards):")
print(all_qualified_teams_2018)


Successfully divided 48 teams into 12 groups (2018 onwards):
  Group A: ['Russia', 'Burkina Faso', 'Colombia', 'Senegal']
  Group B: ['Uzbekistan', 'Netherlands', 'Denmark', 'Saudi Arabia']
  Group C: ['South Korea', 'Morocco', 'Belgium', 'Uruguay']
  Group D: ['Costa Rica', 'Austria', 'Turkey', 'United States']
  Group E: ['Mexico', 'France', 'Japan', 'Cape Verde']
  Group F: ['Egypt', 'Mali', 'Panama', 'Italy']
  Group G: ['Algeria', 'Germany', 'Portugal', 'Poland']
  Group H: ['DR Congo', 'Switzerland', 'England', 'Tunisia']
  Group I: ['Iraq', 'Ivory Coast', 'Croatia', 'United Arab Emirates']
  Group J: ['Australia', 'Norway', 'Ecuador', 'Canada']
  Group K: ['Argentina', 'Spain', 'Ukraine', 'Greece']
  Group L: ['Brazil', 'Nigeria', 'Iran', 'South Africa']

Simulating matches for Group A (2018 onwards):
Standings for Group A (2018 onwards):
  Russia: Points=6, GD=2, GF=7
  Senegal: Points=4, GD=1, GF=5
  Colombia: Points=4, GD=-2, GF=5
  Burkina Faso: Points=3, GD=-1, GF=4

Simula

**Reasoning**:
Now that the `simulate_knockout_match` function is defined and we have the 32 qualified teams from the 2018 filtered data, I will set up the initial pairings for the Round of 32 by shuffling the `all_qualified_teams_2018` list and then simulate each round of the knockout stage (Round of 32, Round of 16, Quarter-finals, Semi-finals, and Final) to determine the ultimate winner of the World Cup using the `elo_ratings_2018`.



In [45]:
import random

# Shuffle the qualified teams for random pairings in Round of 32
random.shuffle(all_qualified_teams_2018)

knockout_results_2018 = {}
current_round_teams_2018 = list(all_qualified_teams_2018)

round_names = ['Round of 32', 'Round of 16', 'Quarter-finals', 'Semi-finals', 'Final']

for round_name in round_names:
    print(f"\n--- {round_name} (2018 onwards) ---")
    winners_of_round_2018 = []
    matches_this_round_2018 = []

    # Pair teams for the current round
    # Assuming an even number of teams for pairing
    for i in range(0, len(current_round_teams_2018), 2):
        team1 = current_round_teams_2018[i]
        team2 = current_round_teams_2018[i+1]

        winner = simulate_knockout_match(team1, team2, elo_ratings_2018)
        winners_of_round_2018.append(winner)

        loser = team1 if winner == team2 else team2
        matches_this_round_2018.append((team1, team2, winner))

        print(f"  {team1} vs {team2} -> Winner: {winner}")

    knockout_results_2018[round_name] = winners_of_round_2018
    current_round_teams_2018 = winners_of_round_2018

    if len(current_round_teams_2018) == 1:
        final_winner_2018 = current_round_teams_2018[0]
        print(f"\n--- World Cup 2026 Winner (2018 onwards): {final_winner_2018} ---")
        break

print("\nKnockout stage simulation complete (2018 onwards).")


--- Round of 32 (2018 onwards) ---
  Norway vs Belgium -> Winner: Belgium
  Russia vs Germany -> Winner: Russia
  Costa Rica vs England -> Winner: England
  Ivory Coast vs United States -> Winner: United States
  Uruguay vs Spain -> Winner: Uruguay
  France vs Nigeria -> Winner: Nigeria
  Iran vs Netherlands -> Winner: Netherlands
  Portugal vs Ukraine -> Winner: Ukraine
  Morocco vs United Arab Emirates -> Winner: Morocco
  Mexico vs Austria -> Winner: Mexico
  Tunisia vs Ecuador -> Winner: Ecuador
  Uzbekistan vs Poland -> Winner: Uzbekistan
  Australia vs Colombia -> Winner: Australia
  Italy vs Brazil -> Winner: Brazil
  Japan vs Senegal -> Winner: Japan
  Croatia vs Mali -> Winner: Mali

--- Round of 16 (2018 onwards) ---
  Belgium vs Russia -> Winner: Russia
  England vs United States -> Winner: England
  Uruguay vs Nigeria -> Winner: Nigeria
  Netherlands vs Ukraine -> Winner: Netherlands
  Morocco vs Mexico -> Winner: Morocco
  Ecuador vs Uzbekistan -> Winner: Uzbekistan
  Aus

## Analyze Colombia's Performance

### Subtask:
Analyze the results of the full tournament simulation to determine how far Colombia advanced and their approximate final standing.


## Summary:

### Q&A
*   **How far did Colombia advance in the simulated tournament?**
    Colombia advanced to the Round of 32 in the knockout stage after qualifying from their group as the 3rd best third-placed team. They were eliminated by Australia in the Round of 32.

*   **What was Colombia's approximate final standing?**
    Colombia finished in the Round of 32, which means they were among the teams ranked 17th to 32nd in the tournament.

### Data Analysis Key Findings
*   The historical match data was successfully filtered to include 7795 matches played from January 1, 2018, onwards.
*   Elo ratings were reinitialized for 229 unique teams and then recalculated based on these filtered matches, with the updated ratings being stored for each match.
*   48 teams were selected for the World Cup 2026, including Colombia, with the remaining 47 teams chosen based on their updated Elo ratings.
*   In the simulated group stage, 48 teams were divided into 12 groups. 24 teams qualified directly as the top two from their respective groups. Additionally, 8 best third-placed teams qualified, with **Colombia** notably securing a spot as the 3rd best third-placed team. This resulted in a total of 32 teams advancing to the knockout stage.
*   In the knockout stage simulation, **Colombia** was eliminated by Australia in the Round of 32. The tournament progressed through the Round of 16, Quarter-finals, and Semi-finals, with England and Morocco reaching the Final. **Morocco** ultimately emerged as the simulated World Cup 2026 winner, defeating England.

### Insights or Next Steps
*   The simulation provides a plausible outcome given the current Elo ratings, highlighting strong contenders like Morocco and England, and identifying potential underdog paths for teams like Colombia to qualify.
*   To refine the simulation, consider running multiple iterations of the full tournament simulation to generate a distribution of outcomes for each team, providing a more robust probabilistic assessment of their chances and final standings.


# Task
Simulate the 2026 World Cup, starting by filtering historical match data from January 1, 2022, recalculating Elo ratings for all teams, selecting 48 World Cup teams (including Colombia) based on these updated ratings, and then simulating both the group and knockout stages to analyze Colombia's performance.

## Update Data Filtering to 2022

### Subtask:
Filter the historical match data to include only matches played from January 1, 2022, onwards, as per the new requirement.


**Reasoning**:
The subtask requires filtering the `df_matches` DataFrame to include only matches played from January 1, 2022, onwards. The 'date' column has already been converted to datetime objects, so I will apply the filter directly and then display the head and info of the new DataFrame as per the instructions.



In [46]:
df_filtered_matches_2022 = df_matches[df_matches['date'] >= '2022-01-01'].copy()
print("Filtered DataFrame head (2022 onwards):")
print(df_filtered_matches_2022.head())
print("\nFiltered DataFrame info (2022 onwards):")
print(df_filtered_matches_2022.info())

Filtered DataFrame head (2022 onwards):
            date     home_team   away_team  home_score  away_score  \
46872 2022-01-01     Indonesia    Thailand           2           2   
46873 2022-01-02         Sudan    Zimbabwe           0           0   
46874 2022-01-02  Burkina Faso       Gabon           3           0   
46875 2022-01-03        Rwanda      Guinea           3           0   
46876 2022-01-04         Gabon  Mauritania           1           1   

                  tournament               country  neutral  outcome  
46872  Southeast Asian Champ             Singapore     True        1  
46873               Friendly              Cameroon     True        1  
46874               Friendly  United Arab Emirates     True        2  
46875               Friendly                Rwanda    False        2  
46876               Friendly  United Arab Emirates     True        1  

Filtered DataFrame info (2022 onwards):
<class 'pandas.core.frame.DataFrame'>
Index: 4250 entries, 46872 to 5112

## Recalculate Elo Ratings

### Subtask:
Reinitialize Elo ratings for all unique teams found in the newly filtered dataset and then recalculate them by iterating through these matches chronologically.


**Reasoning**:
First, I'll extract all unique team names from both the 'home_team' and 'away_team' columns of the `df_filtered_matches_2022` DataFrame and store them in a list. This is necessary to initialize the Elo ratings for all participating teams in this new filtered dataset.



In [47]:
unique_teams_2022 = pd.concat([df_filtered_matches_2022['home_team'], df_filtered_matches_2022['away_team']]).unique()
print(f"Number of unique teams (2022 onwards): {len(unique_teams_2022)}")
print("First 10 unique teams (2022 onwards):")
print(unique_teams_2022[:10])

Number of unique teams (2022 onwards): 225
First 10 unique teams (2022 onwards):
['Indonesia' 'Sudan' 'Burkina Faso' 'Rwanda' 'Gabon' 'Algeria'
 'Cape Verde' 'Cameroon' 'Senegal' 'Morocco']


**Reasoning**:
Now that the unique teams from the 2022 filtered dataset have been identified, I will initialize a dictionary called `elo_ratings_2022` where each unique team is a key and its initial Elo rating is set to 1500, as specified in the instructions.



In [48]:
initial_elo = 1500
elo_ratings_2022 = {team: initial_elo for team in unique_teams_2022}
print(f"Initialized Elo ratings for {len(elo_ratings_2022)} teams (2022 onwards).")
print("Example Elo ratings for first 5 teams (2022 onwards):")
for team in list(elo_ratings_2022.keys())[:5]:
    print(f"{team}: {elo_ratings_2022[team]}")

Initialized Elo ratings for 225 teams (2022 onwards).
Example Elo ratings for first 5 teams (2022 onwards):
Indonesia: 1500
Sudan: 1500
Burkina Faso: 1500
Rwanda: 1500
Gabon: 1500


**Reasoning**:
I will sort the `df_filtered_matches_2022` DataFrame by date, then iterate through each match to recalculate Elo ratings, updating the `elo_ratings_2022` dictionary and adding 'before' and 'after' Elo ratings as new columns to the DataFrame, as specified in the instructions.



In [49]:
df_filtered_matches_2022 = df_filtered_matches_2022.sort_values(by='date').reset_index(drop=True)

# Prepare lists to store new column data
home_elo_before_2022 = []
away_elo_before_2022 = []
home_elo_after_2022 = []
away_elo_after_2022 = []

for index, row in df_filtered_matches_2022.iterrows():
    home_team = row['home_team']
    away_team = row['away_team']
    home_score = row['home_score']
    away_score = row['away_score']

    # Retrieve current Elo ratings
    elo_home = elo_ratings_2022.get(home_team, initial_elo)
    elo_away = elo_ratings_2022.get(away_team, initial_elo)

    # Store 'before match' Elo ratings
    home_elo_before_2022.append(elo_home)
    away_elo_before_2022.append(elo_away)

    # Calculate Elo change using the previously defined function
    change_home, change_away = calculate_elo_change(elo_home, elo_away, home_score, away_score)

    # Update Elo ratings
    new_elo_home = elo_home + change_home
    new_elo_away = elo_away + change_away

    elo_ratings_2022[home_team] = new_elo_home
    elo_ratings_2022[away_team] = new_elo_away

    # Store 'after match' Elo ratings
    home_elo_after_2022.append(new_elo_home)
    away_elo_after_2022.append(new_elo_away)

# Add the new columns to the DataFrame
df_filtered_matches_2022['home_team_elo_before_2022'] = home_elo_before_2022
df_filtered_matches_2022['away_team_elo_before_2022'] = away_elo_before_2022
df_filtered_matches_2022['home_team_elo_after_2022'] = home_elo_after_2022
df_filtered_matches_2022['away_team_elo_after_2022'] = away_elo_after_2022

print("Elo ratings recalculated and updated in df_filtered_matches_2022.")
print("DataFrame head with new Elo columns (2022 onwards):")
print(df_filtered_matches_2022[['date', 'home_team', 'away_team', 'home_team_elo_before_2022', 'away_team_elo_before_2022', 'home_team_elo_after_2022', 'away_team_elo_after_2022']].head())

Elo ratings recalculated and updated in df_filtered_matches_2022.
DataFrame head with new Elo columns (2022 onwards):
        date     home_team   away_team  home_team_elo_before_2022  \
0 2022-01-01     Indonesia    Thailand                     1500.0   
1 2022-01-02         Sudan    Zimbabwe                     1500.0   
2 2022-01-02  Burkina Faso       Gabon                     1500.0   
3 2022-01-03        Rwanda      Guinea                     1500.0   
4 2022-01-04         Gabon  Mauritania                     1485.0   

   away_team_elo_before_2022  home_team_elo_after_2022  \
0                     1500.0                 1500.0000   
1                     1500.0                 1500.0000   
2                     1500.0                 1515.0000   
3                     1500.0                 1515.0000   
4                     1500.0                 1485.6472   

   away_team_elo_after_2022  
0                 1500.0000  
1                 1500.0000  
2                 1485.0000 

## Select World Cup Teams (Including Colombia)

### Subtask:
Select the 48 teams for the World Cup 2026. This selection will ensure Colombia is included, and the remaining teams will be chosen based on their updated Elo ratings.


**Reasoning**:
I will convert the `elo_ratings_2022` dictionary into a Pandas Series, sort it in descending order, ensure 'Colombia' is included, and then select the top 47 teams based on their Elo ratings to form the final 48 World Cup teams.



In [50]:
import pandas as pd

# 1. Convert elo_ratings_2022 dictionary into a Pandas Series
elo_series_2022 = pd.Series(elo_ratings_2022)

# 2. Sort this Series in descending order
sorted_elo_series_2022 = elo_series_2022.sort_values(ascending=False)

# 3. Define 'Colombia' as a mandatory participant and remove it from the ranked list
mandatory_team = 'Colombia'
if mandatory_team in sorted_elo_series_2022:
    remaining_teams_elo_2022 = sorted_elo_series_2022.drop(mandatory_team)
else:
    remaining_teams_elo_2022 = sorted_elo_series_2022.copy()
    print(f"Warning: {mandatory_team} not found in Elo ratings. It will be added manually.")

# 4. Select the top 47 teams from the remaining ranked list
selected_47_teams_2022 = remaining_teams_elo_2022.head(47).index.tolist()

# 5. Combine 'Colombia' with these 47 teams to form the final list of 48 World Cup teams
world_cup_teams_2022 = [mandatory_team] + selected_47_teams_2022

# 6. Print the total number of selected teams and the list of these teams
print(f"Total number of World Cup teams (2022 onwards): {len(world_cup_teams_2022)}")
print("Selected World Cup Teams (2022 onwards):")
for i, team in enumerate(world_cup_teams_2022):
    print(f"  {i+1}. {team}")

Total number of World Cup teams (2022 onwards): 48
Selected World Cup Teams (2022 onwards):
  1. Colombia
  2. Spain
  3. Argentina
  4. Morocco
  5. Japan
  6. Uzbekistan
  7. Senegal
  8. England
  9. South Korea
  10. France
  11. Iran
  12. Portugal
  13. Algeria
  14. Norway
  15. Australia
  16. Netherlands
  17. Mexico
  18. Croatia
  19. Egypt
  20. Brazil
  21. Germany
  22. Russia
  23. Tunisia
  24. Uruguay
  25. United States
  26. Switzerland
  27. Canada
  28. Ecuador
  29. Turkey
  30. Denmark
  31. South Africa
  32. Iraq
  33. Panama
  34. DR Congo
  35. Mali
  36. Italy
  37. Austria
  38. Ivory Coast
  39. Nigeria
  40. Malaysia
  41. Saudi Arabia
  42. Cape Verde
  43. Thailand
  44. United Arab Emirates
  45. Angola
  46. Kosovo
  47. Costa Rica
  48. Greece


**Reasoning**:
I will randomly divide the `world_cup_teams_2022` into 12 groups of 4 teams each as per the instructions. This involves shuffling the teams and then splitting them into groups.



In [51]:
import random

# Shuffle the world_cup_teams_2022 to ensure random grouping
random.shuffle(world_cup_teams_2022)

# Divide teams into 12 groups of 4
groups_2022 = {}
team_index = 0
for i in range(num_groups):
    group_name = f"Group {chr(65 + i)}" # A, B, C, ...
    groups_2022[group_name] = world_cup_teams_2022[team_index : team_index + teams_per_group]
    team_index += teams_per_group

print(f"Successfully divided {len(world_cup_teams_2022)} teams into {num_groups} groups (2022 onwards):")
for group_name, teams in groups_2022.items():
    print(f"  {group_name}: {teams}")

Successfully divided 48 teams into 12 groups (2022 onwards):
  Group A: ['Senegal', 'Costa Rica', 'Mexico', 'Nigeria']
  Group B: ['Ivory Coast', 'United States', 'Argentina', 'Iraq']
  Group C: ['Tunisia', 'Cape Verde', 'South Korea', 'Portugal']
  Group D: ['Canada', 'Colombia', 'Egypt', 'South Africa']
  Group E: ['Denmark', 'France', 'Switzerland', 'Kosovo']
  Group F: ['Panama', 'Algeria', 'Greece', 'Morocco']
  Group G: ['Iran', 'Brazil', 'Norway', 'Angola']
  Group H: ['Australia', 'DR Congo', 'Uruguay', 'Italy']
  Group I: ['Germany', 'Spain', 'Netherlands', 'Russia']
  Group J: ['Croatia', 'England', 'Uzbekistan', 'Malaysia']
  Group K: ['Thailand', 'Austria', 'Japan', 'Mali']
  Group L: ['United Arab Emirates', 'Saudi Arabia', 'Turkey', 'Ecuador']


**Reasoning**:
Now that the groups have been randomly formed, I will iterate through each group, simulate their matches using the `simulate_group_matches` function and the `elo_ratings_2022`, calculate their standings, and then identify the top 2 teams and the 8 best third-placed teams for the knockout stage.



In [52]:
all_group_standings_2022 = {}
for group_name, teams in groups_2022.items():
    print(f"\nSimulating matches for {group_name} (2022 onwards):")
    standings = simulate_group_matches(teams, elo_ratings_2022)
    all_group_standings_2022[group_name] = standings

    print(f"Standings for {group_name} (2022 onwards):")
    # Sort standings for better readability
    sorted_standings = sorted(standings.items(), key=lambda item: (item[1]['points'], item[1]['goal_difference'], item[1]['goals_for']), reverse=True)
    for team, stats in sorted_standings:
        print(f"  {team}: Points={stats['points']}, GD={stats['goal_difference']}, GF={stats['goals_for']}")

print("\nAll group stage matches simulated and standings calculated (2022 onwards).")

# Identify top 2 qualifiers
top_two_qualifiers_2022 = []
for group_name, standings in all_group_standings_2022.items():
    sorted_group_teams = sorted(standings.items(), key=lambda item: (item[1]['points'], item[1]['goal_difference'], item[1]['goals_for']), reverse=True)
    top_two_qualifiers_2022.extend([team for team, stats in sorted_group_teams[:2]])

print(f"\nTotal {len(top_two_qualifiers_2022)} teams qualified directly (top 2 from each group, 2022 onwards).")
print("Directly qualified teams (2022 onwards):")
for i, team in enumerate(top_two_qualifiers_2022):
    print(f"  {i+1}. {team}")

# Identify all third-placed teams
third_placed_teams_2022 = []
for group_name, standings in all_group_standings_2022.items():
    sorted_group_teams = sorted(standings.items(), key=lambda item: (item[1]['points'], item[1]['goal_difference'], item[1]['goals_for']), reverse=True)
    if len(sorted_group_teams) > 2:
        team_name, stats = sorted_group_teams[2]
        third_placed_teams_2022.append({'team': team_name, **stats})

# Sort third-placed teams and select top 8
sorted_third_placed_2022 = sorted(third_placed_teams_2022, key=lambda x: (x['points'], x['goal_difference'], x['goals_for']), reverse=True)
best_third_placed_qualifiers_2022 = [team['team'] for team in sorted_third_placed_2022[:8]]

print(f"\nTotal {len(best_third_placed_qualifiers_2022)} best third-placed teams qualified (2022 onwards).")
print("Best third-placed qualified teams (2022 onwards):")
for i, team in enumerate(best_third_placed_qualifiers_2022):
    print(f"  {i+1}. {team}")

# Combine for final 32 teams
all_qualified_teams_2022 = top_two_qualifiers_2022 + best_third_placed_qualifiers_2022
print(f"\nTotal teams advancing to knockout stage (2022 onwards): {len(all_qualified_teams_2022)}")
print("All qualified teams (2022 onwards):")
print(all_qualified_teams_2022)



Simulating matches for Group A (2022 onwards):
Standings for Group A (2022 onwards):
  Nigeria: Points=6, GD=4, GF=5
  Costa Rica: Points=6, GD=1, GF=4
  Senegal: Points=3, GD=-2, GF=2
  Mexico: Points=3, GD=-3, GF=3

Simulating matches for Group B (2022 onwards):
Standings for Group B (2022 onwards):
  Iraq: Points=6, GD=2, GF=5
  Ivory Coast: Points=6, GD=1, GF=5
  Argentina: Points=6, GD=1, GF=3
  United States: Points=0, GD=-4, GF=0

Simulating matches for Group C (2022 onwards):
Standings for Group C (2022 onwards):
  South Korea: Points=9, GD=3, GF=5
  Cape Verde: Points=4, GD=1, GF=3
  Portugal: Points=3, GD=-2, GF=2
  Tunisia: Points=1, GD=-2, GF=4

Simulating matches for Group D (2022 onwards):
Standings for Group D (2022 onwards):
  Egypt: Points=6, GD=2, GF=5
  South Africa: Points=6, GD=0, GF=5
  Colombia: Points=3, GD=0, GF=6
  Canada: Points=3, GD=-2, GF=3

Simulating matches for Group E (2022 onwards):
Standings for Group E (2022 onwards):
  Kosovo: Points=9, GD=3, GF=4

**Reasoning**:
Now that the `simulate_knockout_match` function is defined and we have the 32 qualified teams from the 2022 filtered data, I will set up the initial pairings for the Round of 32 by shuffling the `all_qualified_teams_2022` list and then simulate each round of the knockout stage (Round of 32, Round of 16, Quarter-finals, Semi-finals, and Final) to determine the ultimate winner of the World Cup using the `elo_ratings_2022`.



In [53]:
import random

# Shuffle the qualified teams for random pairings in Round of 32
random.shuffle(all_qualified_teams_2022)

knockout_results_2022 = {}
current_round_teams_2022 = list(all_qualified_teams_2022)

round_names = ['Round of 32', 'Round of 16', 'Quarter-finals', 'Semi-finals', 'Final']

for round_name in round_names:
    print(f"\n--- {round_name} (2022 onwards) ---")
    winners_of_round_2022 = []
    matches_this_round_2022 = []

    # Pair teams for the current round
    # Assuming an even number of teams for pairing
    for i in range(0, len(current_round_teams_2022), 2):
        team1 = current_round_teams_2022[i]
        team2 = current_round_teams_2022[i+1]

        winner = simulate_knockout_match(team1, team2, elo_ratings_2022)
        winners_of_round_2022.append(winner)

        loser = team1 if winner == team2 else team2
        matches_this_round_2022.append((team1, team2, winner))

        print(f"  {team1} vs {team2} -> Winner: {winner}")

    knockout_results_2022[round_name] = winners_of_round_2022
    current_round_teams_2022 = winners_of_round_2022

    if len(current_round_teams_2022) == 1:
        final_winner_2022 = current_round_teams_2022[0]
        print(f"\n--- World Cup 2026 Winner (2022 onwards): {final_winner_2022} ---")
        break

print("\nKnockout stage simulation complete (2022 onwards).")


--- Round of 32 (2022 onwards) ---
  Spain vs Uruguay -> Winner: Spain
  Malaysia vs South Korea -> Winner: South Korea
  Switzerland vs Costa Rica -> Winner: Costa Rica
  Netherlands vs Norway -> Winner: Netherlands
  England vs South Africa -> Winner: England
  Russia vs Morocco -> Winner: Morocco
  Colombia vs Ecuador -> Winner: Ecuador
  Argentina vs Saudi Arabia -> Winner: Argentina
  DR Congo vs Japan -> Winner: Japan
  Iraq vs Kosovo -> Winner: Iraq
  Thailand vs United Arab Emirates -> Winner: United Arab Emirates
  Uzbekistan vs Brazil -> Winner: Uzbekistan
  Egypt vs Nigeria -> Winner: Nigeria
  Ivory Coast vs Greece -> Winner: Greece
  Austria vs Denmark -> Winner: Denmark
  Cape Verde vs Italy -> Winner: Italy

--- Round of 16 (2022 onwards) ---
  Spain vs South Korea -> Winner: Spain
  Costa Rica vs Netherlands -> Winner: Netherlands
  England vs Morocco -> Winner: Morocco
  Ecuador vs Argentina -> Winner: Ecuador
  Japan vs Iraq -> Winner: Japan
  United Arab Emirates vs

## Analyze Colombia's Performance

### Subtask:
Analyze the results of the full tournament simulation to determine how far Colombia advanced and their approximate final standing.

## Summary:

### Q&A
*   **How far did Colombia advance in the simulated tournament?**
    Colombia advanced to the Round of 32 in the knockout stage after qualifying from their group as the 3rd best third-placed team. They were eliminated by Ecuador in the Round of 32.

*   **What was Colombia's approximate final standing?**
    Colombia finished in the Round of 32, which means they were among the teams ranked 17th to 32nd in the tournament.

### Data Analysis Key Findings
*   The historical match data was successfully filtered to include 4250 matches played from January 1, 2022, onwards.
*   Elo ratings were reinitialized for 225 unique teams and then recalculated based on these filtered matches, with the updated ratings being stored for each match.
*   48 teams were selected for the World Cup 2026, including Colombia, with the remaining 47 teams chosen based on their updated Elo ratings.
*   In the simulated group stage, 48 teams were divided into 12 groups. 24 teams qualified directly as the top two from their respective groups. Additionally, 8 best third-placed teams qualified, with **Colombia** notably securing a spot as the 3rd best third-placed team.
*   In the knockout stage simulation, **Colombia** was eliminated by Ecuador in the Round of 32.
*   The tournament progressed through the Round of 16, Quarter-finals, and Semi-finals, with Netherlands and Italy reaching the Final. **Italy** ultimately emerged as the simulated World Cup 2026 winner, defeating Netherlands.

### Insights or Next Steps
*   Colombia's performance was consistent across the different data filtering periods, consistently qualifying for the knockout stages, though often as a best third-placed team. This suggests they are a competitive team but may struggle against stronger opponents in later stages.
*   To gain a more statistically robust understanding of team performance and tournament outcomes, it would be beneficial to run this full simulation multiple times (e.g., 1,000 or 10,000 iterations) and then average the results to account for the stochastic nature of the match simulations. This would provide a clearer picture of Colombia's average advancement and potential.


## Define World Cup 2026 Structure

### Subtask:
Reconfirm the structure of the World Cup 2026, including the number of groups and the progression rules for both group and knockout stages, as this forms the framework for the simulation.


**Reasoning**:
To confirm that the World Cup structure variables are correctly set and accessible, I will print their values.



In [54]:
print(f"Number of World Cup groups: {num_groups}")
print(f"Teams per group: {teams_per_group}")
print("Group stage progression rules:")
for rule in group_stage_progression_rules:
    print(f"- {rule}")
print("Knockout stage progression rules:")
for rule in knockout_stage_progression_rules:
    print(f"- {rule}")

Number of World Cup groups: 12
Teams per group: 4
Group stage progression rules:
- Top 2 teams from each of the 12 groups (24 teams) automatically advance.
- 8 best third-placed teams across all groups also qualify.
- Total of 32 teams advancing to the knockout stage.
Knockout stage progression rules:
- Round of 32
- Round of 16
- Quarter-finals
- Semi-finals
- Final
- Standard single-elimination rules apply.


## Summary:

### Q&A
*   **How far did Colombia advance in the simulated tournament?**
    Colombia advanced to the Round of 32 in the knockout stage after qualifying from their group as the 3rd best third-placed team. They were eliminated by Ecuador in the Round of 32.

*   **What was Colombia's approximate final standing?**
    Colombia finished in the Round of 32, which means they were among the teams ranked 17th to 32nd in the tournament.

### Data Analysis Key Findings
*   The historical match data was successfully filtered to include 4250 matches played from January 1, 2022, onwards.
*   Elo ratings were reinitialized for 225 unique teams and then recalculated based on these filtered matches, with the updated ratings being stored for each match.
*   48 teams were selected for the World Cup 2026, including Colombia, with the remaining 47 teams chosen based on their updated Elo ratings.
*   The World Cup 2026 structure was defined with 12 groups of 4 teams. Progression rules stated that the top 2 teams from each group and the 8 best third-placed teams would qualify for the knockout stage (total 32 teams).
*   In the simulated group stage, 24 teams qualified directly as the top two from their respective groups. Additionally, 8 best third-placed teams qualified, with **Colombia** notably securing a spot as the 3rd best third-placed team.
*   In the knockout stage simulation, **Colombia** was eliminated by Ecuador in the Round of 32.
*   The tournament progressed through the Round of 16, Quarter-finals, and Semi-finals, with Netherlands and Italy reaching the Final. **Italy** ultimately emerged as the simulated World Cup 2026 winner, defeating Netherlands.

### Insights or Next Steps
*   Colombia's performance was consistent across the different data filtering periods, consistently qualifying for the knockout stages, though often as a best third-placed team. This suggests they are a competitive team but may struggle against stronger opponents in later stages.
*   To gain a more statistically robust understanding of team performance and tournament outcomes, it would be beneficial to run this full simulation multiple times (e.g., 1,000 or 10,000 iterations) and then average the results to account for the stochastic nature of the match simulations. This would provide a clearer picture of Colombia's average advancement and potential.


# Task
Simulate the 2026 World Cup, starting by filtering historical match data from January 1, 2024, recalculating Elo ratings for all teams, selecting 48 World Cup teams (including Colombia) based on these updated ratings, and then simulating both the group and knockout stages to analyze Colombia's performance.

## Update Data Filtering to 2024

### Subtask:
Filter the historical match data to include only matches played from January 1, 2024, onwards, as per the new requirement.


**Reasoning**:
The subtask requires filtering the `df_matches` DataFrame to include only matches played from January 1, 2024, onwards. The 'date' column has already been converted to datetime objects, so I will apply the filter directly and then display the head and info of the new DataFrame as per the instructions.



In [55]:
df_filtered_matches_2024 = df_matches[df_matches['date'] >= '2024-01-01'].copy()
print("Filtered DataFrame head (2024 onwards):")
print(df_filtered_matches_2024.head())
print("\nFiltered DataFrame info (2024 onwards):")
print(df_filtered_matches_2024.info())

Filtered DataFrame head (2024 onwards):
            date     home_team  away_team  home_score  away_score tournament  \
48921 2024-01-01         Japan   Thailand           5           0   Friendly   
48922 2024-01-01     Hong Kong      China           2           1   Friendly   
48923 2024-01-02         Libya  Indonesia           4           0   Friendly   
48924 2024-01-04  Saudi Arabia    Lebanon           1           0   Friendly   
48925 2024-01-04    Tajikistan  Hong Kong           2           1   Friendly   

                    country  neutral  outcome  
48921                 Japan    False        2  
48922  United Arab Emirates     True        2  
48923                Turkey     True        2  
48924                 Qatar     True        2  
48925  United Arab Emirates     True        2  

Filtered DataFrame info (2024 onwards):
<class 'pandas.core.frame.DataFrame'>
Index: 2201 entries, 48921 to 51121
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype      

**Reasoning**:
First, I'll extract all unique team names from both the 'home_team' and 'away_team' columns of the `df_filtered_matches_2024` DataFrame and store them in a list. This is necessary to initialize the Elo ratings for all participating teams in this new filtered dataset.



In [56]:
unique_teams_2024 = pd.concat([df_filtered_matches_2024['home_team'], df_filtered_matches_2024['away_team']]).unique()
print(f"Number of unique teams (2024 onwards): {len(unique_teams_2024)}")
print("First 10 unique teams (2024 onwards):")
print(unique_teams_2024[:10])

Number of unique teams (2024 onwards): 220
First 10 unique teams (2024 onwards):
['Japan' 'Hong Kong' 'Libya' 'Saudi Arabia' 'Tajikistan' 'Iran'
 'Kyrgyzstan' 'Qatar' 'Togo' 'Australia']


**Reasoning**:
Now that the unique teams from the 2024 filtered dataset have been identified, I will initialize a dictionary called `elo_ratings_2024` where each unique team is a key and its initial Elo rating is set to 1500, as specified in the instructions.



In [57]:
initial_elo = 1500
elo_ratings_2024 = {team: initial_elo for team in unique_teams_2024}
print(f"Initialized Elo ratings for {len(elo_ratings_2024)} teams (2024 onwards).")
print("Example Elo ratings for first 5 teams (2024 onwards):")
for team in list(elo_ratings_2024.keys())[:5]:
    print(f"{team}: {elo_ratings_2024[team]}")

Initialized Elo ratings for 220 teams (2024 onwards).
Example Elo ratings for first 5 teams (2024 onwards):
Japan: 1500
Hong Kong: 1500
Libya: 1500
Saudi Arabia: 1500
Tajikistan: 1500


**Reasoning**:
I will sort the `df_filtered_matches_2024` DataFrame by date, then iterate through each match to recalculate Elo ratings, updating the `elo_ratings_2024` dictionary and adding 'before' and 'after' Elo ratings as new columns to the DataFrame, as specified in the instructions.



In [58]:
df_filtered_matches_2024 = df_filtered_matches_2024.sort_values(by='date').reset_index(drop=True)

# Prepare lists to store new column data
home_elo_before_2024 = []
away_elo_before_2024 = []
home_elo_after_2024 = []
away_elo_after_2024 = []

for index, row in df_filtered_matches_2024.iterrows():
    home_team = row['home_team']
    away_team = row['away_team']
    home_score = row['home_score']
    away_score = row['away_score']

    # Retrieve current Elo ratings
    elo_home = elo_ratings_2024.get(home_team, initial_elo)
    elo_away = elo_ratings_2024.get(away_team, initial_elo)

    # Store 'before match' Elo ratings
    home_elo_before_2024.append(elo_home)
    away_elo_before_2024.append(elo_away)

    # Calculate Elo change using the previously defined function
    change_home, change_away = calculate_elo_change(elo_home, elo_away, home_score, away_score)

    # Update Elo ratings
    new_elo_home = elo_home + change_home
    new_elo_away = elo_away + change_away

    elo_ratings_2024[home_team] = new_elo_home
    elo_ratings_2024[away_team] = new_elo_away

    # Store 'after match' Elo ratings
    home_elo_after_2024.append(new_elo_home)
    away_elo_after_2024.append(new_elo_away)

# Add the new columns to the DataFrame
df_filtered_matches_2024['home_team_elo_before_2024'] = home_elo_before_2024
df_filtered_matches_2024['away_team_elo_before_2024'] = away_elo_before_2024
df_filtered_matches_2024['home_team_elo_after_2024'] = home_elo_after_2024
df_filtered_matches_2024['away_team_elo_after_2024'] = away_elo_after_2024

print("Elo ratings recalculated and updated in df_filtered_matches_2024.")
print("DataFrame head with new Elo columns (2024 onwards):")
print(df_filtered_matches_2024[['date', 'home_team', 'away_team', 'home_team_elo_before_2024', 'away_team_elo_before_2024', 'home_team_elo_after_2024', 'away_team_elo_after_2024']].head())

Elo ratings recalculated and updated in df_filtered_matches_2024.
DataFrame head with new Elo columns (2024 onwards):
        date     home_team  away_team  home_team_elo_before_2024  \
0 2024-01-01         Japan   Thailand                     1500.0   
1 2024-01-01     Hong Kong      China                     1500.0   
2 2024-01-02         Libya  Indonesia                     1500.0   
3 2024-01-04  Saudi Arabia    Lebanon                     1500.0   
4 2024-01-04    Tajikistan  Hong Kong                     1500.0   

   away_team_elo_before_2024  home_team_elo_after_2024  \
0                     1500.0                 1515.0000   
1                     1500.0                 1515.0000   
2                     1500.0                 1515.0000   
3                     1500.0                 1515.0000   
4                     1515.0                 1515.6472   

   away_team_elo_after_2024  
0                 1485.0000  
1                 1485.0000  
2                 1485.0000  
3   

**Reasoning**:
I will convert the `elo_ratings_2024` dictionary into a Pandas Series, sort it in descending order, ensure 'Colombia' is included, and then select the top 47 teams based on their Elo ratings to form the final 48 World Cup teams, as per the instructions.



In [59]:
import pandas as pd

# 1. Convert elo_ratings_2024 dictionary into a Pandas Series
elo_series_2024 = pd.Series(elo_ratings_2024)

# 2. Sort this Series in descending order
sorted_elo_series_2024 = elo_series_2024.sort_values(ascending=False)

# 3. Define 'Colombia' as a mandatory participant and remove it from the ranked list
mandatory_team = 'Colombia'
if mandatory_team in sorted_elo_series_2024:
    remaining_teams_elo_2024 = sorted_elo_series_2024.drop(mandatory_team)
else:
    remaining_teams_elo_2024 = sorted_elo_series_2024.copy()
    print(f"Warning: {mandatory_team} not found in Elo ratings. It will be added manually.")

# 4. Select the top 47 teams from the remaining ranked list
selected_47_teams_2024 = remaining_teams_elo_2024.head(47).index.tolist()

# 5. Combine 'Colombia' with these 47 teams to form the final list of 48 World Cup teams
world_cup_teams_2024 = [mandatory_team] + selected_47_teams_2024

# 6. Print the total number of selected teams and the list of these teams
print(f"Total number of World Cup teams (2024 onwards): {len(world_cup_teams_2024)}")
print("Selected World Cup Teams (2024 onwards):")
for i, team in enumerate(world_cup_teams_2024):
    print(f"  {i+1}. {team}")

Total number of World Cup teams (2024 onwards): 48
Selected World Cup Teams (2024 onwards):
  1. Colombia
  2. Spain
  3. Morocco
  4. Japan
  5. Senegal
  6. Argentina
  7. Uzbekistan
  8. England
  9. South Korea
  10. Algeria
  11. Norway
  12. Iran
  13. South Africa
  14. Australia
  15. Germany
  16. France
  17. DR Congo
  18. Tunisia
  19. Russia
  20. Netherlands
  21. Kosovo
  22. Nigeria
  23. Switzerland
  24. Mexico
  25. Croatia
  26. Portugal
  27. Canada
  28. Angola
  29. Saudi Arabia
  30. Egypt
  31. Brazil
  32. Denmark
  33. Ivory Coast
  34. Costa Rica
  35. Mali
  36. Curaçao
  37. Thailand
  38. Italy
  39. United States
  40. Cape Verde
  41. Austria
  42. Haiti
  43. Panama
  44. Turkey
  45. Iraq
  46. Ecuador
  47. Malaysia
  48. Cameroon


## Simulate Group Stage Matches

### Subtask:
Simulate all matches in the group stage using the newly calculated Elo ratings, determine group standings, and identify which teams advance to the knockout rounds.

## Simulate Group Stage Matches

### Subtask:
Simulate all matches in the group stage using the newly calculated Elo ratings, determine group standings, and identify which teams advance to the knockout rounds.

**Reasoning**:
Now that the `world_cup_teams_2024` have been selected, I will randomly divide them into 12 groups of 4 teams each, as per the defined World Cup structure, to prepare for the group stage simulation.



In [60]:
import random

# Shuffle the world_cup_teams_2024 to ensure random grouping
random.shuffle(world_cup_teams_2024)

# Divide teams into 12 groups of 4
groups_2024 = {}
team_index = 0
for i in range(num_groups):
    group_name = f"Group {chr(65 + i)}" # A, B, C, ...
    groups_2024[group_name] = world_cup_teams_2024[team_index : team_index + teams_per_group]
    team_index += teams_per_group

print(f"Successfully divided {len(world_cup_teams_2024)} teams into {num_groups} groups (2024 onwards):")
for group_name, teams in groups_2024.items():
    print(f"  {group_name}: {teams}")

Successfully divided 48 teams into 12 groups (2024 onwards):
  Group A: ['Croatia', 'Netherlands', 'Morocco', 'Iraq']
  Group B: ['Tunisia', 'Thailand', 'France', 'Kosovo']
  Group C: ['England', 'Cameroon', 'South Korea', 'Mexico']
  Group D: ['Uzbekistan', 'Iran', 'Spain', 'Panama']
  Group E: ['Russia', 'Germany', 'Turkey', 'Ivory Coast']
  Group F: ['Senegal', 'Algeria', 'Australia', 'Cape Verde']
  Group G: ['Denmark', 'Canada', 'Costa Rica', 'Italy']
  Group H: ['Japan', 'Malaysia', 'Haiti', 'Ecuador']
  Group I: ['Colombia', 'Egypt', 'Saudi Arabia', 'DR Congo']
  Group J: ['Mali', 'Angola', 'Switzerland', 'Nigeria']
  Group K: ['Portugal', 'South Africa', 'Norway', 'United States']
  Group L: ['Brazil', 'Austria', 'Curaçao', 'Argentina']


**Reasoning**:
Now that the groups have been randomly formed, I will iterate through each group, simulate their matches using the `simulate_group_matches` function and the `elo_ratings_2024`, calculate their standings, and then identify the top 2 teams and the 8 best third-placed teams for the knockout stage.



In [61]:
all_group_standings_2024 = {}
for group_name, teams in groups_2024.items():
    print(f"\nSimulating matches for {group_name} (2024 onwards):")
    standings = simulate_group_matches(teams, elo_ratings_2024)
    all_group_standings_2024[group_name] = standings

    print(f"Standings for {group_name} (2024 onwards):")
    # Sort standings for better readability
    sorted_standings = sorted(standings.items(), key=lambda item: (item[1]['points'], item[1]['goal_difference'], item[1]['goals_for']), reverse=True)
    for team, stats in sorted_standings:
        print(f"  {team}: Points={stats['points']}, GD={stats['goal_difference']}, GF={stats['goals_for']}")

print("\nAll group stage matches simulated and standings calculated (2024 onwards).")

# Identify top 2 qualifiers
top_two_qualifiers_2024 = []
for group_name, standings in all_group_standings_2024.items():
    sorted_group_teams = sorted(standings.items(), key=lambda item: (item[1]['points'], item[1]['goal_difference'], item[1]['goals_for']), reverse=True)
    top_two_qualifiers_2024.extend([team for team, stats in sorted_group_teams[:2]])

print(f"\nTotal {len(top_two_qualifiers_2024)} teams qualified directly (top 2 from each group, 2024 onwards).")
print("Directly qualified teams (2024 onwards):")
for i, team in enumerate(top_two_qualifiers_2024):
    print(f"  {i+1}. {team}")

# Identify all third-placed teams
third_placed_teams_2024 = []
for group_name, standings in all_group_standings_2024.items():
    sorted_group_teams = sorted(standings.items(), key=lambda item: (item[1]['points'], item[1]['goal_difference'], item[1]['goals_for']), reverse=True)
    if len(sorted_group_teams) > 2:
        team_name, stats = sorted_group_teams[2]
        third_placed_teams_2024.append({'team': team_name, **stats})

# Sort third-placed teams and select top 8
sorted_third_placed_2024 = sorted(third_placed_teams_2024, key=lambda x: (x['points'], x['goal_difference'], x['goals_for']), reverse=True)
best_third_placed_qualifiers_2024 = [team['team'] for team in sorted_third_placed_2024[:8]]

print(f"\nTotal {len(best_third_placed_qualifiers_2024)} best third-placed teams qualified (2024 onwards).")
print("Best third-placed qualified teams (2024 onwards):")
for i, team in enumerate(best_third_placed_qualifiers_2024):
    print(f"  {i+1}. {team}")

# Combine for final 32 teams
all_qualified_teams_2024 = top_two_qualifiers_2024 + best_third_placed_qualifiers_2024
print(f"\nTotal teams advancing to knockout stage (2024 onwards): {len(all_qualified_teams_2024)}")
print("All qualified teams (2024 onwards):")
print(all_qualified_teams_2024)


Simulating matches for Group A (2024 onwards):
Standings for Group A (2024 onwards):
  Morocco: Points=7, GD=4, GF=6
  Croatia: Points=6, GD=1, GF=4
  Iraq: Points=4, GD=1, GF=6
  Netherlands: Points=0, GD=-6, GF=0

Simulating matches for Group B (2024 onwards):
Standings for Group B (2024 onwards):
  France: Points=7, GD=4, GF=5
  Kosovo: Points=4, GD=1, GF=4
  Tunisia: Points=3, GD=-2, GF=2
  Thailand: Points=3, GD=-3, GF=3

Simulating matches for Group C (2024 onwards):
Standings for Group C (2024 onwards):
  Mexico: Points=6, GD=3, GF=5
  South Korea: Points=4, GD=-1, GF=4
  Cameroon: Points=4, GD=-1, GF=2
  England: Points=2, GD=-1, GF=2

Simulating matches for Group D (2024 onwards):
Standings for Group D (2024 onwards):
  Spain: Points=6, GD=2, GF=6
  Panama: Points=6, GD=2, GF=6
  Uzbekistan: Points=6, GD=1, GF=3
  Iran: Points=0, GD=-5, GF=1

Simulating matches for Group E (2024 onwards):
Standings for Group E (2024 onwards):
  Turkey: Points=6, GD=2, GF=4
  Russia: Points=6,

## Simulate Knockout Stage Matches

### Subtask:
Simulate the matches from the Round of 32 up to the final, based on the outcomes of the group stage and using the updated Elo ratings for predictions.


**Reasoning**:
Now that the `simulate_knockout_match` function is defined and we have the 32 qualified teams from the 2024 filtered data, I will set up the initial pairings for the Round of 32 by shuffling the `all_qualified_teams_2024` list and then simulate each round of the knockout stage (Round of 32, Round of 16, Quarter-finals, Semi-finals, and Final) to determine the ultimate winner of the World Cup using the `elo_ratings_2024`.



In [62]:
import random

# Shuffle the qualified teams for random pairings in Round of 32
random.shuffle(all_qualified_teams_2024)

knockout_results_2024 = {}
current_round_teams_2024 = list(all_qualified_teams_2024)

round_names = ['Round of 32', 'Round of 16', 'Quarter-finals', 'Semi-finals', 'Final']

for round_name in round_names:
    print(f"\n--- {round_name} (2024 onwards) ---")
    winners_of_round_2024 = []
    matches_this_round_2024 = []

    # Pair teams for the current round
    # Assuming an even number of teams for pairing
    for i in range(0, len(current_round_teams_2024), 2):
        team1 = current_round_teams_2024[i]
        team2 = current_round_teams_2024[i+1]

        winner = simulate_knockout_match(team1, team2, elo_ratings_2024)
        winners_of_round_2024.append(winner)

        loser = team1 if winner == team2 else team2
        matches_this_round_2024.append((team1, team2, winner))

        print(f"  {team1} vs {team2} -> Winner: {winner}")

    knockout_results_2024[round_name] = winners_of_round_2024
    current_round_teams_2024 = winners_of_round_2024

    if len(current_round_teams_2024) == 1:
        final_winner_2024 = current_round_teams_2024[0]
        print(f"\n--- World Cup 2026 Winner (2024 onwards): {final_winner_2024} ---")
        break

print("\nKnockout stage simulation complete (2024 onwards).")


--- Round of 32 (2024 onwards) ---
  Ivory Coast vs Colombia -> Winner: Ivory Coast
  Egypt vs Denmark -> Winner: Denmark
  Russia vs Haiti -> Winner: Russia
  Saudi Arabia vs Morocco -> Winner: Saudi Arabia
  Nigeria vs Cape Verde -> Winner: Nigeria
  Algeria vs Japan -> Winner: Algeria
  Croatia vs France -> Winner: France
  Kosovo vs Mali -> Winner: Kosovo
  Uzbekistan vs Italy -> Winner: Italy
  Portugal vs Mexico -> Winner: Mexico
  South Korea vs Cameroon -> Winner: South Korea
  Curaçao vs United States -> Winner: Curaçao
  Austria vs Turkey -> Winner: Turkey
  Ecuador vs Costa Rica -> Winner: Costa Rica
  Iraq vs Panama -> Winner: Iraq
  Spain vs Switzerland -> Winner: Switzerland

--- Round of 16 (2024 onwards) ---
  Ivory Coast vs Denmark -> Winner: Ivory Coast
  Russia vs Saudi Arabia -> Winner: Russia
  Nigeria vs Algeria -> Winner: Algeria
  France vs Kosovo -> Winner: Kosovo
  Italy vs Mexico -> Winner: Mexico
  South Korea vs Curaçao -> Winner: Curaçao
  Turkey vs Costa

## Analyze Colombia's Performance

### Subtask:
Analyze the results of the full tournament simulation to determine how far Colombia advanced and their approximate final standing.

## Summary:

### Q&A
*   **How far did Colombia advance in the simulated tournament (2024 onwards)?**
    Colombia advanced to the Round of 32 in the knockout stage after qualifying from their group as a direct qualifier (top 2 team). They were eliminated by Ivory Coast in the Round of 32.

*   **What was Colombia's approximate final standing (2024 onwards)?**
    Colombia finished in the Round of 32, which means they were among the teams ranked 17th to 32nd in the tournament.

### Data Analysis Key Findings
*   The historical match data was successfully filtered to include 2201 matches played from January 1, 2024, onwards.
*   Elo ratings were reinitialized for 220 unique teams and then recalculated based on these filtered matches.
*   48 teams were selected for the World Cup 2026, including Colombia, with the remaining 47 teams chosen based on their updated Elo ratings.
*   In the simulated group stage, **Colombia** finished second in Group I, successfully qualifying directly for the knockout stage.
*   In the knockout stage simulation, **Colombia** was eliminated by Ivory Coast in the Round of 32.
*   The tournament progressed through the various knockout rounds, with Russia ultimately emerging as the simulated World Cup 2026 winner, defeating Switzerland in the final.

### Insights or Next Steps
*   Colombia's performance in this simulation, based on the most recent data (2024 onwards), shows them qualifying directly from the group stage, which is an improvement over previous simulations where they qualified as a best third-placed team. However, they were still eliminated in the Round of 32, indicating a consistent challenge in advancing further in the knockout rounds.
*   To gain a more statistically robust understanding of team performance and tournament outcomes, it would be beneficial to run this full simulation multiple times (e.g., 1,000 or 10,000 iterations) and then average the results to account for the stochastic nature of the match simulations. This would provide a clearer picture of Colombia's average advancement and potential across different filtering periods.

## Analyze Colombia's Performance

### Subtask:
Analyze the results of the full tournament simulation to determine how far Colombia advanced and their approximate final standing.


## Summary:

### Q&A
*   **How far did Colombia advance in the simulated tournament (2024 onwards)?**
    Colombia advanced to the Round of 32 in the knockout stage after qualifying from their group as a direct qualifier (top 2 team). They were eliminated by Ivory Coast in the Round of 32.

*   **What was Colombia's approximate final standing (2024 onwards)?**
    Colombia finished in the Round of 32, which means they were among the teams ranked 17th to 32nd in the tournament.

### Data Analysis Key Findings
*   Historical match data was successfully filtered to include 2201 matches played from January 1, 2024, onwards.
*   Elo ratings were reinitialized for 220 unique teams identified from the filtered 2024 matches and then recalculated based on this recent data.
*   A total of 48 teams were selected for the World Cup, including Colombia as a mandatory participant, along with the top 47 teams based on their final Elo ratings.
*   In the simulated group stage, Colombia finished second in Group I with 4 points, a goal difference of -1, and 2 goals for, thereby qualifying directly for the knockout stage.
*   In the simulated knockout stage, Colombia was eliminated in the Round of 32 after losing to Ivory Coast.
*   The simulation concluded with Russia winning the World Cup 2026, defeating Switzerland in the final.

### Insights or Next Steps
*   Colombia's performance in this simulation, based on the most recent data (2024 onwards), shows them qualifying directly from the group stage, which is an improvement over previous simulations where they qualified as a best third-placed team. However, they were still eliminated in the Round of 32, indicating a consistent challenge in advancing further in the knockout rounds.
*   To gain a more statistically robust understanding of team performance and tournament outcomes, it would be beneficial to run this full simulation multiple times (e.g., 1,000 or 10,000 iterations) and then average the results to account for the stochastic nature of the match simulations. This would provide a clearer picture of Colombia's average advancement and potential across different filtering periods.
